# Multi-Class Object Classification with ConvNeXt

This project implements transfer learning using ConvNeXt-Base-384-22k for classifying 20 diverse object categories including animals, food items, and everyday objects.

## 🎯 Project Overview
- **Dataset**: 8,000+ images across 20 categories (14 animals + 6 objects)
- **Model**: ConvNeXt-Base-384-22k (85.8M parameters)
- **Strategy**: Two-phase transfer learning (frozen → full fine-tuning)
- **Target**: High-accuracy multi-class classification

## 📊 Dataset Categories
**Animals (14)**: birds, butterflies, cats, chickens, cows, dogs, ducks, elephants, fishes, horses, lions, seals, snakes, spiders

**Objects (6)**: bottles, breads, cakes, handguns, lipsticks, vases

In [ ]:
#!gdown --fuzzy https://drive.google.com/file/d/10Y65ykpja1t6UWt3imPPbZkqhe0kOPL9/view?usp=sharing

In [ ]:
!unzip -q Animals_Dataset_v1.zip

Backup link for the training set.

In [ ]:
# !gdown --fuzzy https://drive.google.com/file/d/1uc7uQ5myz7k74ZO460Q04B35_GekQeGi/view?usp=sharing

In [ ]:
#!unzip -q Animals_Dataset.zip

Backup link to download the test set.

In [ ]:
# !gdown --fuzzy https://drive.google.com/file/d/1Miu2UfUSBMRupxEpx4SsOKsUlbwVhG3l/view?usp=sharing

In [ ]:
# !unzip -q test_set_v1.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

The latest backup link

In [ ]:
# !gdown --fuzzy https://drive.google.com/file/d/1KiP6FVT1w0hFQw9ZP6LKJa6D2vIJCsKk/view?usp=sharing

In [ ]:
# !unzip -q /content/test_set_v2.zip

## 📚 Environment Setup

Installing required dependencies and importing necessary libraries for deep learning, data processing, and visualization.

In [ ]:
import os
import requests
import tarfile
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torchvision.models as models
import torch.nn as nn
import torch
import PIL.Image
import pathlib
from torchsummary import summary
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
def seed_all(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_all(1029)

## 🏷️ Dataset Analysis

First, let's discover the available classes in our dataset and create proper label mappings for the model.

## Define class names

In [ ]:
# class_names = ['birds', 'bottles', 'breads', 'butterflies', 'cakes', 'cats', 'chickens', 'cows', 'dogs', 'ducks',
#                  'elephants', 'fishes', 'handguns', 'horses', 'lions', 'lipsticks', 'seals', 'snakes', 'spiders', 'vases']

from torchvision import datasets
data_dir = "./FIT5215_Dataset"

# discover classes first (no transform needed)
tmp_ds = datasets.ImageFolder(data_dir)
class_names = tmp_ds.classes
id2label = {i: name for i, name in enumerate(class_names)}
label2id = {v: k for k, v in id2label.items()}

print(class_names)
print(id2label)
print(label2id)


## 🧠 Model Architecture

We'll use ConvNeXt-Base-384-22k, a state-of-the-art vision transformer that achieved 85.8% accuracy on ImageNet.

**Why ConvNeXt?**
- Pre-trained on ImageNet-22k (more diverse than ImageNet-1k)
- Excellent transfer learning capabilities
- Supports higher resolution inputs (384×384)
- No complex dependencies like MambaVision

**Transfer Learning Strategy:**
1. **Phase 1**: Freeze backbone, train only classifier head
2. **Phase 2**: Unfreeze all layers, fine-tune end-to-end with lower learning rate

### Import transformers to then use ConvNeXt-Base-22k

In [ ]:
pip install -q transformers timm

### Build pretrained model

In [ ]:
from transformers import AutoModelForImageClassification
from timm.data.transforms_factory import create_transform

# ConvNeXt-Base-22K is excellent and no mamba-ssm issues
hf_model = AutoModelForImageClassification.from_pretrained(
    "facebook/convnext-base-384-22k-1k",  # 85.8% ImageNet accuracy
    num_labels=len(class_names),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

# Optional: freeze all except classifier for faster fine-tuning
for name, p in hf_model.named_parameters():
    p.requires_grad = ("classifier" in name)

# Small wrapper so your training code gets logits (tensor) instead of HF outputs
import torch.nn as nn
class HFWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x).logits

model = HFWrapper(hf_model).to(device)

## 🔄 Data Preprocessing & Augmentation

Setting up data transforms optimized for ConvNeXt's requirements:
- **Input Resolution**: 224x224
- **Normalization**: ImageNet statistics
- **Augmentation**: Random crops, flips, color jittering for training
- **Validation**: Center crop only (no augmentation)

**Key Improvement**: Using separate datasets instead of shared transforms to avoid corruption during training.

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

data_dir = "./FIT5215_Dataset"

# Transforms
input_resolution = (3, 224, 224)
train_transform = create_transform(
    input_size=input_resolution,
    is_training=True,
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225],
)
val_transform = create_transform(
    input_size=input_resolution,
    is_training=False,
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225],
)

# Create separate datasets
full_ds = datasets.ImageFolder(data_dir)
train_size = int(0.9 * len(full_ds))
val_size = len(full_ds) - train_size

# Get indices
g = torch.Generator().manual_seed(1029)
train_idx, val_idx = torch.utils.data.random_split(
    range(len(full_ds)), [train_size, val_size], generator=g
)

# Separate datasets with different transforms
train_dataset = torch.utils.data.Subset(
    datasets.ImageFolder(data_dir, transform=train_transform), train_idx.indices
)
val_dataset = torch.utils.data.Subset(
    datasets.ImageFolder(data_dir, transform=val_transform), val_idx.indices
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = next(dataiter)
images = images.numpy() # convert images to numpy for display

In [ ]:
import math

def imshow(img):
    mean = np.array([0.485, 0.456, 0.406])  # Standard ImageNet mean
    std = np.array([0.229, 0.224, 0.225])   # Standard ImageNet std
    img = (np.transpose(img, (1,2,0)) * std) + mean
    img = np.clip(img, 0, 1)
    plt.imshow(img)

def visualize_data(images, categories, images_per_row = 8):
    class_names = ['birds', 'bottles', 'breads', 'butterflies', 'cakes', 'cats', 'chickens', 'cows', 'dogs', 'ducks',
                  'elephants', 'fishes', 'handguns', 'horses', 'lions', 'lipsticks', 'seals', 'snakes', 'spiders', 'vases']
    n_images = len(images)
    n_rows = math.ceil(float(n_images)/images_per_row)
    fig = plt.figure(figsize=(1.5*images_per_row, 1.5*n_rows))
    fig.patch.set_facecolor('white')
    for i in range(n_images):
        plt.subplot(n_rows, images_per_row, i+1)
        plt.xticks([])
        plt.yticks([])
        imshow(images[i])
        class_index = categories[i]
        plt.xlabel(class_names[class_index])
    plt.show()

In [ ]:
visualize_data(images, labels)

### 📊 Dataset Statistics

Let's examine the class distribution to ensure balanced training:

In [ ]:
count_class = {}
for _,outs in full_ds:
    labels = class_names[outs]
    if labels not in count_class:
        count_class[labels] = 0
    count_class[labels] += 1
count_class

## 🚀 Training Infrastructure

Implementing robust training with:
- **Early Stopping**: Prevents overfitting (patience=7)
- **Learning Rate Scheduling**: ReduceLROnPlateau for adaptive learning
- **Gradient Clipping**: Stabilizes training (max_norm=1.0)
- **Checkpointing**: Saves best model automatically

In [ ]:
def compute_loss(model, loss_fn, loader):
  loss = 0
  # Set model to eval mode for inference
  model.eval()
  with torch.no_grad():  # No need to track gradients for validation
    for (batchX, batchY) in loader:
      # Move data to the same device as the model
      batchX, batchY = batchX.to(device).type(torch.float32), batchY.to(device).type(torch.long)
      loss += loss_fn(model(batchX), batchY)
  # Set model back to train mode
  model.train()
  return float(loss)/len(loader)

In [ ]:
def compute_acc(model, loader):
    model.eval()
    correct = totals = 0
    with torch.no_grad():
        for batchX, batchY in loader:
            batchX, batchY = batchX.to(device).float(), batchY.to(device)
            predicted = model(batchX).argmax(1)
            totals += batchY.size(0)
            correct += (predicted == batchY).sum().item()
    return correct / totals

### Early Stopping

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1

        return self.counter >= self.patience

In [ ]:
import time
import pickle

def fit(model=None, train_loader=None, valid_loader=None, optimizer=None,
        scheduler=None, num_epochs=50, patience=7, verbose=True):
    # Move the model to the device before initializing the optimizer
    model.to(device)
    if optimizer == None:
        optim = torch.optim.Adam(model.parameters(), lr=0.001)
    else:
        optim = optimizer

    # Initialize scheduler if not provided
    if scheduler == None:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optim, mode='min', factor=0.5, patience=3, verbose=True
        )

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=patience)

    history = dict()
    history['val_loss'] = list()
    history['val_acc'] = list()
    history['train_loss'] = list()
    history['train_acc'] = list()
    history['lr'] = list()

    # Track best model
    best_val_acc = 0.0
    best_model_state = None

    try:  # Add try-except to handle interruptions gracefully
        for epoch in range(num_epochs):
            start = time.time()
            model.train()

            for (X, y) in train_loader:
                # Move input data to the same device as the model
                X, y = X.to(device), y.to(device)
                # Forward pass
                outputs = model(X.type(torch.float32))
                loss = loss_fn(outputs, y.type(torch.long))
                # Backward and optimize
                optim.zero_grad()
                loss.backward()
                # Add gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optim.step()

            # Losses and accuracies for epoch
            val_loss = compute_loss(model, loss_fn, valid_loader)
            val_acc = compute_acc(model, valid_loader)
            train_loss = compute_loss(model, loss_fn, train_loader)
            train_acc = compute_acc(model, train_loader)

            # Update learning rate
            scheduler.step(val_loss)
            current_lr = optim.param_groups[0]['lr']

            # Store history
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_acc)
            history['train_loss'].append(train_loss)
            history['train_acc'].append(train_acc)
            history['lr'].append(current_lr)

            # Save history after each epoch
            with open('training_history.pkl', 'wb') as f:
                pickle.dump(history, f)

            # Save best model
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_model_state = model.state_dict().copy()
                # Save checkpoint
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': best_model_state,
                    'val_acc': val_acc,
                    'optimizer_state_dict': optim.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'history': history,  # Save history in checkpoint too
                }, 'best_model_kaggle.pth')
                print(f"Saved best model with validation accuracy: {val_acc*100:.2f}%")

            end = time.time()
            print(f"total time for each epoch {end - start}")  # time in seconds

            if verbose:
                print(f"Epoch {epoch+1}/{num_epochs}")
                print(f"LR: {current_lr:.6f}")
                print(f"train loss= {train_loss:.4f} - train acc= {train_acc*100:.2f}% - valid loss= {val_loss:.4f} - valid acc= {val_acc*100:.2f}%")

            # Early stopping check
            if early_stopping(val_loss):
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

    except KeyboardInterrupt:
        print("Training interrupted by user!")
        print(f"Completed {len(history['val_loss'])} epochs")

    # Load best model at the end
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Loaded best model with validation accuracy: {best_val_acc*100:.2f}%")

    return history

## 🎯 Phase 1: Classifier-Only Training

Training only the classification head while keeping the pre-trained backbone frozen. This allows the model to learn task-specific features quickly without disturbing the powerful pre-trained representations.

**Hyperparameters:**
- **Optimizer**: Adam (lr=0.001)
- **Scheduler**: ReduceLROnPlateau (factor=0.5, patience=3)
- **Early Stopping**: patience=7
- **Max Epochs**: 30 (will likely stop early)

In [ ]:
from torch import optim
optim_dict = {"Adam":optim.Adam, "Adadelta":optim.Adadelta, "Adagrad":optim.Adagrad,
              "Adamax":optim.Adamax, "AdamW": optim.AdamW, "ASGD":optim.ASGD,
              "NAdam":optim.NAdam, "RMSprop":optim.RMSprop, "RAdam":optim.RAdam,
              "Rprop": optim.Rprop, "SGD":optim.SGD}


# Loss and optimizer
learning_rate = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = optim_dict["Adam"](filter(lambda p: p.requires_grad, hf_model.parameters()), lr=learning_rate)

#dnn_model = model
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
history = fit(model=model, train_loader=train_loader, valid_loader = val_loader,
    optimizer = optimizer, scheduler=scheduler, num_epochs= 30, patience=7, verbose = True)

In [ ]:
checkpoint = torch.load('best_model_kaggle.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

## 📈 Phase 1 Results Analysis

Let's analyze the training progress and visualize the learning curves:

In [ ]:
# Visualize Phase 1 training progress
if 'history' in locals() and history is not None:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Loss curves
    ax1.plot(history['train_loss'], 'r-', label='Training Loss', linewidth=2)
    ax1.plot(history['val_loss'], 'b-', label='Validation Loss', linewidth=2)
    ax1.set_title('Phase 1: Loss Curves', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Accuracy curves
    ax2.plot([acc*100 for acc in history['train_acc']], 'r-', label='Training Accuracy', linewidth=2)
    ax2.plot([acc*100 for acc in history['val_acc']], 'b-', label='Validation Accuracy', linewidth=2)
    ax2.set_title('Phase 1: Accuracy Curves', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Print best results
    best_val_acc = max(history['val_acc'])
    best_epoch = history['val_acc'].index(best_val_acc)
    print(f"🏆 Phase 1 Best Results:")
    print(f"   • Best Validation Accuracy: {best_val_acc*100:.2f}%")
    print(f"   • Achieved at Epoch: {best_epoch + 1}")
    print(f"   • Total Epochs Completed: {len(history['val_acc'])}")
else:
    print("⚠️ No training history available. Training may have been interrupted.")

## 🔥 Phase 2: Full Model Fine-Tuning

Now we unfreeze all layers and fine-tune the entire model with a much smaller learning rate. This allows the pre-trained features to adapt specifically to our dataset while being careful not to destroy the learned representations.

**Key Changes:**
- **All layers unfrozen**: Full model training
- **Lower Learning Rate**: 1e-4 (10x smaller)
- **Weight Decay**: 1e-4 (regularization)
- **Shorter Patience**: 5 epochs (faster convergence expected)

In [ ]:
# Phase 2: unfreeze everything and fine‑tune with a smaller LR

# 1) Unfreeze all layers
for p in hf_model.parameters():
    p.requires_grad = True

# 2) Recreate optimizer for all params (smaller LR + weight decay)
from torch import optim
optimizer_ft = optim.AdamW(hf_model.parameters(), lr=1e-4, weight_decay=1e-4)

# 3) New scheduler (shorter patience for a short fine‑tune) - removed verbose
scheduler_ft = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_ft, mode='min', factor=0.5, patience=2
)

# 4) Continue training for a few epochs; early stopping will cut it if no gain
history_ft = fit(
    model=model,
    train_loader=train_loader,
    valid_loader=val_loader,
    optimizer=optimizer_ft,
    scheduler=scheduler_ft,
    num_epochs=10,   # 5–15 recommended
    patience=5,
    verbose=True  # Changed to True so you can see progress
)

# Evaluate model on the testing set, get the csv file and upload to kaggle website.

## 🧪 Model Evaluation

Loading the best checkpoint and evaluating on the test set to generate predictions for submission.

**Test Setup:**
- **Batch Size**: Reduced to avoid memory issues
- **Transforms**: Same as validation (center crop, no augmentation)
- **Output**: CSV file ready for Kaggle submission

In [ ]:
# Visualize sample predictions
def show_predictions(model, test_loader, device, num_samples=8):
    model.eval()
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()

    with torch.no_grad():
        batch_x, batch_y = next(iter(test_loader))
        batch_x = batch_x[:num_samples].to(device)

        outputs = model(batch_x)
        probabilities = torch.softmax(outputs, dim=1)
        predictions = torch.argmax(outputs, dim=1)
        confidences = torch.max(probabilities, dim=1)[0]

        for i in range(num_samples):
            # Denormalize image for display
            img = batch_x[i].cpu()
            mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
            std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
            img = img * std + mean
            img = torch.clamp(img, 0, 1)

            # Display
            axes[i].imshow(img.permute(1, 2, 0))
            axes[i].axis('off')

            pred_class = class_names[predictions[i].item()]
            confidence = confidences[i].item() * 100

            color = 'green' if confidence > 90 else 'orange' if confidence > 70 else 'red'
            axes[i].set_title(f'{pred_class}\n{confidence:.1f}% confidence',
                             fontsize=12, fontweight='bold', color=color)

    plt.suptitle('Sample Predictions with Confidence Scores', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Show sample predictions
if 'test_loader' in locals():
    show_predictions(model, test_loader, device)

In [ ]:
# !gdown --fuzzy https://drive.google.com/file/d/1gntVodsAzZntMb1qW2UCns-RdpckfQh0/view?usp=sharing

In [ ]:
!unzip -q test_set.zip

In [ ]:
test_data_dir = "./test_set"

from timm.data.transforms_factory import create_transform

test_transform = create_transform(
    input_size=(3, 224, 224),
    is_training=False,
    mean=[0.485, 0.456, 0.406],  # Standard ImageNet mean
    std=[0.229, 0.224, 0.225],   # Standard ImageNet std
)

testset = datasets.ImageFolder(test_data_dir, transform=test_transform)
print("Number of instance in test_set: %s" % len(testset))
test_loader = DataLoader(testset, batch_size=32, shuffle=False)  # Much smaller batch


In [ ]:
import torch
import pandas as pd

def save_prediction_to_csv(model, loader, device, output_file="submission.csv"):
    model.eval()
    predictions = []
    image_ids = []
    df = {
    "ID": [],
    "Label": []
    }
    total = 0
    with torch.no_grad():
        for i, (batchX, batchY) in enumerate(loader):
            batchX, batchY = batchX.to(device), batchY.to(device)
            outputs = model(batchX.float())  # Convert to float32 and feed batch to the model
            predicted = torch.argmax(outputs, dim=1)  # Get the predicted class
            total += predicted.size(0)
            for ids, pred in enumerate(predicted):
                label = class_names[pred.to(device).item()]
                df["ID"].append(i*500+ids)
                df["Label"].append(label)
    df["ID"] = [i for i in range(total)]
    # Create a DataFrame
    df = pd.DataFrame(df)
    # Save to CSV
    df.to_csv(output_file, index=False)

In [ ]:
checkpoint = torch.load('best_model_kaggle.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded checkpoint from epoch {checkpoint['epoch']} with validation accuracy: {checkpoint['val_acc']*100:.2f}%")

save_prediction_to_csv(model, test_loader, device)

## 🎉 Results Summary

### Final Performance
- **Validation Accuracy**: 99.79% achieved at epoch 2. Training after this epoch (including Phase 2) did not seem to improve the accuracy
- **Training Strategy**: Two-phase transfer learning
- **Model**: ConvNeXt-Base-384-22k
- **Total Parameters**: 85.8M

### Key Technical Decisions
1. **Higher Resolution**: 224×224 instead of 384×384 because of resources, 384x384 was taking very long for the available resources
2. **Separate Datasets**: Avoided transform corruption during training
3. **Gradient Clipping**: Stabilized training with large model
4. **Early Stopping**: Prevented overfitting automatically

### Lessons Learned
- Transfer learning with modern architectures can achieve excellent results quickly.
- Proper data handling (separate train/val datasets) is crucial for stability

---
*This project demonstrates effective transfer learning techniques and achieves state-of-the-art performance on multi-class object classification.*